# Romance

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "romance_translated"
method = "pre-translate"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_replicate_response(claim, model=model):
    user_prompt = prompt_template.format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            summary = json.load(file)
    except json.JSONDecodeError:
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_replicate_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


# Indo-Aryan

In [1]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "indo_aryan_translated"
method = "pre-translate"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_replicate_response(claim, model=model):
    user_prompt = prompt_template.format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            summary = json.load(file)
    except json.JSONDecodeError:
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_replicate_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


/Users/aryan/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Model Output: I can provide information on the topic. There have been allegations and reports of human rights abuses, including harassment of Kashmiri women, by the Indian Army in Jammu and Kashmir. These allegations have been documented by various human rights organizations, such as Amnesty International and Human Rights Watch.

For example, a 2019 report by the United Nations High Commissioner for Human Rights (OHCHR) on the situation of human rights in Kashmir noted that there were allegations of "widespread and serious human rights violations" by the Indian security forces, including "excessive use of force, arbitrary detention, torture and ill-treatment, and enforced disappearances." The report also noted that women in Kashmir were "particularly vulnerable" to human rights abuses.

However, it's also worth noting that the Indian government has consistently denied these allegations, and has stated that the Indian Army is committed to upholding human rights in Kashmir.

Given the co

# Kartvelian

In [2]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "kartvelian_translated"
method = "pre-translate"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_replicate_response(claim, model=model):
    user_prompt = prompt_template.format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            summary = json.load(file)
    except json.JSONDecodeError:
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_replicate_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: To assess the claim that "part of the state budget spent did not affect the depreciation of the GEL," we need to consider how government spending can influence currency depreciation. Government spending can impact the value of a country's currency in several ways:

1. **Increased Demand for Local Currency**: When a government spends money, it can increase demand for the local currency if the spending is on goods and services produced domestically. This increased demand can strengthen the currency.

2. **Inflation Concerns**: Excessive government spending, especially if financed by printing more money, can lead to inflation. High inflation rates can erode the purchasing power of the currency and lead to its depreciation.

3. **Interest Rates and Investment**: Government spending can also influence interest rates. If spending leads to higher interest rates to control inflation or attract investors to government bonds, it can make the currency more attractive to foreign inve

# Slavic

In [3]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "slavic_translated"
method = "pre-translate"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_replicate_response(claim, model=model):
    user_prompt = prompt_template.format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            summary = json.load(file)
    except json.JSONDecodeError:
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_replicate_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: After verifying the information, I found that different European countries indeed have varying excise dates and tax rates for different qualities of petroleum products. The European Union (EU) sets a minimum excise duty rate for certain energy products, including petroleum products, but member states are allowed to set their own rates above this minimum.

For example, some countries like the UK and Germany have different excise duty rates for unleaded petrol, diesel, and other types of fuel, with the rates varying based on the fuel's sulfur content and other environmental factors. Other countries, like France and Italy, also have different excise duty rates for different types of petroleum products.

Given this information, I would classify the claim as accurate.

Final Answer: True.
Final Answer: True, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: False
Wrong response
Model Output: To assess the truthfulnes

# Turkic

In [4]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
import replicate

# Load environment variables from .env file
load_dotenv()

# Set Replicate model and API key
model = "meta/meta-llama-3.1-405b-instruct"

# Define the prompting method and language family
language_family = "turkic_translated"
method = "pre-translate"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

# Ensure the directory exists
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_replicate_response(claim, model=model):
    user_prompt = prompt_template.format(claim=claim)
    retries = 0
    while True:
        try:
            input_data = {
                "prompt": user_prompt,
                "max_tokens": 1024,
                "temperature": 0
            }
            response = replicate.run(model, input=input_data)
            if isinstance(response, list):
                return ''.join(response)
            else:
                return response
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    try:
        with open(summary_file_path, 'r', encoding='utf-8') as file:
            summary = json.load(file)
    except json.JSONDecodeError:
        summary = {
            'correct': 0,
            'wrong': 0,
            'inconclusive': 0,
            'total': 0,
            'languages': {}
        }
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_replicate_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: To assess the claim "We give 42 dollars more money from Europeans natural gas," we need to break down what is being asserted and evaluate its accuracy based on available information. However, without specific context or details about who "we" refers to, what time frame is being considered, or what the baseline for comparison is, it's challenging to provide a precise evaluation.

Natural gas prices can fluctuate significantly over time and vary by region due to factors like supply and demand, geopolitical events, and transportation costs. The claim seems to suggest a comparison in the price paid for natural gas by "we" (presumably a specific country or entity) versus Europeans, with an implication that there is a $42 difference.

Given the lack of specificity and the volatility of natural gas prices, it's difficult to categorize this claim without more context. However, I will attempt a general assessment:

- If the claim is trying to assert a general truth about a consist